In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import json
import random,time
from random import randint
import re
import urllib
import sys
from urllib import request
from datetime import datetime
import pandas as pd
from pandas.core.frame import DataFrame
from datetime import datetime
mydir='cookpad'
mydir_save_img="cookpad_save_img"
mydir_save_log='cookpad_log'
def create_mkdir():
    if os.path.exists(mydir):
        print('mrdir name cookpad has been create')
    else:
        os.mkdir(mydir)

    if os.path.exists(mydir_save_img):
        print("mrdir name cookpad_save_img has been create")
    else:
        os.mkdir(mydir_save_img)
    
    if os.path.exists(mydir_save_log):
        print("mrdir name cookpad_save_log has been create")
    else:
        os.mkdir(mydir_save_log)

def write_to_file(write_file_list):
    with open('./cookpad/'+save_name+'.txt','w',encoding="utf-8") as es:
        es.write('食譜名稱:{}'.format(title_name)+'|'+'\n')
        es.write('作者:{}'.format(arthor)+'|'+'\n')
        es.write('圖片:{}'.format(img)+'|'+'\n')
        es.write('份數:{}'.format(Number_of_copies)+'|'+'\n')
        es.write('料理時間:{}'.format(cook_time)+'|'+'\n')
        es.write('食材:{}'.format(set_food)+'|'+'\n')
        es.write('食材數量:{}'.format(set_food_count)+'|'+'\n')
        es.write('料理步驟:{}'.format(set_step)+'|'+'\n')
        print('寫檔完成')
def get_main_body(url):
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:75.0) Gecko/20100101 Firefox/75.0'}
    ss=requests.session()
    res=ss.get(url,headers=headers)
    cookies=ss.cookies.get_dict()
    #print(ss.cookies.get_dict())

    res=ss.get(url,headers=headers,cookies=cookies)
    soup=BeautifulSoup(res.text,'html.parser')
    return soup
def save_img(img):
    if img!="na":
        print("進入寫入圖片前")
        request.urlretrieve(img, './cookpad_save_img/{}.jpg'.format(save_name))#寫入圖片
        print("圖片下載完成")
    else:
        print("image not found")
def save_log(set_next_url):
    with open('./cookpad_log/'+str(set_next_url)+'.txt','w',encoding="utf-8") as es2:
        es2.write("網址{}".format(getnextpageurl)+'\n')
    

    
def count_time():
    time_strat=time.time()
    #time.sleep(5)
    time.sleep(randint(5,15))
    time_end=time.time()
    print(time_end-time_strat)
def list_to_csv(list_save):
    data=DataFrame(list_save)#这时候是以行为标准写入的
    print(data)
    now = datetime.now()
    s1 = datetime.strftime(now,'%Y-%m-%d')
    s2= datetime.strftime(now,'%H:%M:%S')
    s2=s2.replace(":","_")
    #print (type(now))
    #print (now)
    #print (s1)
    #print(s2)
    save_file_data_time=s1+"_"+s2
    #print(s.replace(" ","")))
    data.to_csv('cookpad{}.csv'.format(save_file_data_time),header=False,index=False,encoding='utf-8-sig')


create_mkdir()
#一些計數器變數    
url="https://cookpad.com/tw"
set_next_url_count=1
set_next_url=1
set_total_items=1
set_while_stop=1
list_save=[['食譜名稱', '作者','圖片','份數', '料理時間', '食材','食材數量', '料理步驟']]
tep_list=[]
save_next_html=[]#存下檢查爬到哪裡了
set_food=''
set_step=''
log_list_save=[['編號','網站']]   
while (set_while_stop==1):
#for go  in  range(1,3):
    get_main_body(url)    
    soup=get_main_body(url)
    #print(soup)    
    mainbody=soup.select('div.media')
    getnextpageurl="https://cookpad.com"+soup.select('div#feed_pagination')[0].select('a')[0]['href']
    save_next_html.append(getnextpageurl)#紀錄html
    

    for i in mainbody:
        
        print("第{}筆資料".format(set_total_items))
        title=i.select('div.media__body')[0].select('a.flex-grow')[0]['aria-label']
        test2=i.select('div.media__body')[0].select('a.flex-grow')[0]
        title_url="https://cookpad.com"+i.select('div.media__body')[0].select('a.flex-grow')[0]['href']
        url=title_url
        get_main_body(url)
        soup=get_main_body(url)
        #print(soup)
    
        print(title)
        #print(test2)
        print(title_url)
        count_time()
        #如果img沒有抓到
        img=soup.select('div.tofu_image')
        if len(img)<=0:
            print("img is not found")
            img="na"
        else:
            img=img[0].select('img')[0]['src']
            split_img=img.split(':')[1]#進行切片防止:被轉碼
            test=urllib.parse.quote(split_img)#網址中文轉碼
            img="https:"+test#進行網址重組
        title_name=soup.select('section.intro-container')[0].select('h1.break-words')[0].text
        #artice=soup.select('div.recipe-show__story')[0].select('p.mb-2')[0].text.split('#')[0]
        
        arthor=soup.select('section.document__section.media.author-container')[0].select('span.text-cookpad-16')[0].text
        cook_time=soup.select('div.p-3.border-t.border-quaternary.text-center.text-secondary')
        if len(cook_time) <= 0 :
            cook_time = "Na"
            print("cook_tine is na")
        else:
            cook_time = cook_time[0].select('span.px-4')[0].text #有些資料沒有
           
        food_test=soup.select('div.ingredient-list')[0].select('ol.list-unstyled.text-cookpad-18')[0].select('li')
        Number_of_copies=soup.select('div.text-secondary.mt-2')[0].text
        if len(Number_of_copies.split())<=0:
            Number_of_copies='na'
        else:
            print('Number_of_copies is alive')
    
            
        step=soup.select('ol.numbered-list')[0].select('p.mb-2')
            
           
        step_number=1
        set_food=''
        set_step=''
        set_food_count=''
        
        for x in food_test:
            get_food=x.select('div')[0].text
            #print(get_food)
            get_food=get_food.split(' ')[8]
            #print(get_food)
            get_food_count=x.select('div')[0].select('bdi')[0].text
            set_food=set_food+get_food+'\n'
            set_food_count=set_food_count+get_food_count+"\n"
       
        for z in step:
            get_step=z.text
            set_step=set_step+str(step_number)+"."+get_step
            step_number=step_number+1
        
        step_number=1#重設步驟變數
        #food_total_and_food_step(food_test,step,step_number)
        
        #print(img)
        #print(title_name)
        #print(artice)
        #print(Awards)
        #print(arthor)
        #print(cook_time)
        #print(food_total)
        #print(set_food)
        #print(set_step)
        #print(img)
        #print(Number_of_copies)
        #存入資料
        title_name=title_name.replace(" ","")
        set_food=set_food.replace(" ","")
            
        tep_list.append(title_name)
        tep_list.append(arthor)
        tep_list.append(img)
        tep_list.append(Number_of_copies)
        tep_list.append(cook_time)
        tep_list.append(set_food)
        tep_list.append(set_food_count)
        tep_list.append(set_step)
        list_save.append(tep_list)
        
        #print(list_save)
        tep_list=[]
        # write_to_file(img, content, cook, cook_time)
           
        #print(list_save)
        #print("_____________________________________")
            
        #print("正規化檔名前")
        #print("_____________________________________")
        string=title_name
        re_title_name=re.sub(u"([^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a])","",string)
        #print(re_title_name)
        save_name_id=str(set_total_items)
        save_name=save_name_id+"_"+re_title_name
        #print(save_name)
        #print("正規化檔名後")
            
            
            
            
        #print("_____________________________________")
        
        #print(入寫檔前')
        
        write_file_list=[title_name, arthor, img, Number_of_copies,cook_time,set_food,set_food_count,set_step]
        write_to_file(write_file_list)
            
        save_img(img)    
 
                
           
            
        print("_____________________________________")
            
        
      
        set_total_items+=1

    #寫入next_URL_LOG       
    save_log(set_next_url)
    set_next_url+=1
    
    url=getnextpageurl
    print("+++++++換頁+++++++++++++++++")
    print("第{}個頁面".format(set_next_url_count))
    
    print(getnextpageurl)
    set_next_url_count+=1
"""
    if set_total_items==1001:
        set_while_stop=set_while_stop-1
        print("已經有一千筆資料了")  
"""    

if getnextpageurl=='https://cookpad.com/tw?after=1526305209.0':
    set_while_stop=set_while_stop-1
    
    

print(set_total_items-1)

list_to_csv(list_save)




第1筆資料
涼拌蛋沙拉 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12936774-%E6%B6%BC%E6%8B%8C%E8%9B%8B%E6%B2%99%E6%8B%89
12.000671625137329
cook_tine is na
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第2筆資料
病毒蛋炒飯【Covid-19 fried rice】 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12936779-%E7%97%85%E6%AF%92%E8%9B%8B%E7%82%92%E9%A3%AFcovid-19-fried-rice
10.000072240829468
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第3筆資料
鮭魚炒麵 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12885034-%E9%AE%AD%E9%AD%9A%E7%82%92%E9%BA%B5
5.000061273574829
cook_tine is na
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第4筆資料
「香菇培根蛋」健康的食材原味 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12930088-%E3%80%8C%E9%A6%99%E8%8F%87%E5%9F%B9%E6%A0%B9%E8%9B%8B%E3%80%8D%E5%81%A5%E5%BA%B7%E7%9A%84%E9%A3%9F%E6%9D%90%E5%8E%9F%E5%91%B3
6.000006198883057
cook_tine is na
寫檔完成
進入寫入圖片前
圖片下載完成
______________________________

汆燙小管 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12912024-%E6%B1%86%E7%87%99%E5%B0%8F%E7%AE%A1
13.000114440917969
cook_tine is na
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第37筆資料
【影片】韓國網紅-焦糖脆脆咖啡拿鐵 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12931093-%E5%BD%B1%E7%89%87%E9%9F%93%E5%9C%8B%E7%B6%B2%E7%B4%85-%E7%84%A6%E7%B3%96%E8%84%86%E8%84%86%E5%92%96%E5%95%A1%E6%8B%BF%E9%90%B5
14.000330924987793
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第38筆資料
雞肉豆腐煲 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12930824-%E9%9B%9E%E8%82%89%E8%B1%86%E8%85%90%E7%85%B2
5.0000762939453125
cook_tine is na
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第39筆資料
咸肉粽 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12930618-%E5%92%B8%E8%82%89%E7%B2%BD
14.000600576400757
cook_tine is na
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第40筆資料
【 南瓜意大利粉】： link
https://cookpad.com/tw/%E9

9.000075340270996
cook_tine is na
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第70筆資料
簡單控醣 生酮 原味豆皮納豆披薩 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12874752-%E7%B0%A1%E5%96%AE%E6%8E%A7%E9%86%A3-%E7%94%9F%E9%85%AE-%E5%8E%9F%E5%91%B3%E8%B1%86%E7%9A%AE%E7%B4%8D%E8%B1%86%E6%8A%AB%E8%96%A9
14.000423669815063
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第71筆資料
炸餛飩 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12926559-%E7%82%B8%E9%A4%9B%E9%A3%A9
8.000315189361572
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第72筆資料
芒果 Ice Cream 🍦 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12921920-%E8%8A%92%E6%9E%9C-ice-cream-%F0%9F%8D%A6
9.000057935714722
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第73筆資料
手作生煎包 Pan Fried Dumpling link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12926488-%E6%89%8B%E4%BD%9C%E7%94%9F%E7%85%8E%E5%8C%85-pan-fried-dumpling
14.000737428665161
cook_

15.000104665756226
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第104筆資料
番茄盅 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12921529-%E7%95%AA%E8%8C%84%E7%9B%85
8.00046682357788
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第105筆資料
青椒牛肉炒飯 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12921614-%E9%9D%92%E6%A4%92%E7%89%9B%E8%82%89%E7%82%92%E9%A3%AF
8.000033378601074
cook_tine is na
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第106筆資料
黑糖蜜糖吐司♥ link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12921586-%E9%BB%91%E7%B3%96%E8%9C%9C%E7%B3%96%E5%90%90%E5%8F%B8%E2%99%A5
10.000457763671875
cook_tine is na
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第107筆資料
高麗菜紐澳良雞丁捲 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12921402-%E9%AB%98%E9%BA%97%E8%8F%9C%E7%B4%90%E6%BE%B3%E8%89%AF%E9%9B%9E%E4%B8%81%E6%8D%B2
8.00076150894165
cook_tine is na
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第108筆資料
洋蔥炒蛋 link
https://cookpa

起司烤香菇 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/11746359-%E8%B5%B7%E5%8F%B8%E7%83%A4%E9%A6%99%E8%8F%87
13.00033688545227
cook_tine is na
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第138筆資料
蔥油鮮蚵菜豆粥 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12918438-%E8%94%A5%E6%B2%B9%E9%AE%AE%E8%9A%B5%E8%8F%9C%E8%B1%86%E7%B2%A5
10.000638723373413
cook_tine is na
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第139筆資料
酥皮起司素熱狗 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12918231-%E9%85%A5%E7%9A%AE%E8%B5%B7%E5%8F%B8%E7%B4%A0%E7%86%B1%E7%8B%97
6.000503063201904
cook_tine is na
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第140筆資料
蝦仁肉絲蛋炒飯 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12917775-%E8%9D%A6%E4%BB%81%E8%82%89%E7%B5%B2%E8%9B%8B%E7%82%92%E9%A3%AF
11.000041007995605
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第141筆資料
彩色珍珠丸子 link
https://cookpad.com/tw/%E9%A3%9F

...

圖片下載完成
_____________________________________
第2113筆資料
清燉獅子頭 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12714651-%E6%B8%85%E7%87%89%E7%8D%85%E5%AD%90%E9%A0%AD
8.000163555145264
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第2114筆資料
麻婆雞蛋豆腐 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12621108-%E9%BA%BB%E5%A9%86%E9%9B%9E%E8%9B%8B%E8%B1%86%E8%85%90
15.0000581741333
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第2115筆資料
蒸豆瓣排骨 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12620481-%E8%92%B8%E8%B1%86%E7%93%A3%E6%8E%92%E9%AA%A8
6.000548839569092
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第2116筆資料
馬鈴薯炒肉絲 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12714481-%E9%A6%AC%E9%88%B4%E8%96%AF%E7%82%92%E8%82%89%E7%B5%B2
12.001568078994751
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第2117筆資料
桑椹蘋果優酪乳 link
https://cookpad.com/tw/%E9%A3%9F%E8%A

蛤礪獨享鍋 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12711506-%E8%9B%A4%E7%A4%AA%E7%8D%A8%E4%BA%AB%E9%8D%8B
6.000831842422485
cook_tine is na
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第2148筆資料
蘋果派（減糖版） link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12711422-%E8%98%8B%E6%9E%9C%E6%B4%BE%E6%B8%9B%E7%B3%96%E7%89%88
15.000843048095703
cook_tine is na
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第2149筆資料
清燒瓢瓜 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/9555768-%E6%B8%85%E7%87%92%E7%93%A2%E7%93%9C
5.000228643417358
cook_tine is na
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第2150筆資料
雞蛋豆腐（自製） link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12711220-%E9%9B%9E%E8%9B%8B%E8%B1%86%E8%85%90%E8%87%AA%E8%A3%BD
7.001033306121826
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
+++++++換頁+++++++++++++++++
第104個頁面
https://cookpad.com/tw?after=1590827259.0
第2151筆資料
乾煎香魚 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12711255-%E

圖片下載完成
_____________________________________
第2182筆資料
韓式焦糖餅 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12706936-%E9%9F%93%E5%BC%8F%E7%84%A6%E7%B3%96%E9%A4%85
7.000338315963745
cook_tine is na
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第2183筆資料
南瓜薯仔炆雞翼#壓力煲 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12706275-%E5%8D%97%E7%93%9C%E8%96%AF%E4%BB%94%E7%82%86%E9%9B%9E%E7%BF%BC%E5%A3%93%E5%8A%9B%E7%85%B2
9.000043630599976
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第2184筆資料
蜂蜜蛋糕 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12705540-%E8%9C%82%E8%9C%9C%E8%9B%8B%E7%B3%95
14.000194072723389
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第2185筆資料
三種材料搞定煉乳餅乾 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12706159-%E4%B8%89%E7%A8%AE%E6%9D%90%E6%96%99%E6%90%9E%E5%AE%9A%E7%85%89%E4%B9%B3%E9%A4%85%E4%B9%BE
14.00007438659668
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
___________________________________

麻婆豆腐 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12702679-%E9%BA%BB%E5%A9%86%E8%B1%86%E8%85%90
9.000795364379883
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第2217筆資料
懶人版涼拌芽菜 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12702529-%E6%87%B6%E4%BA%BA%E7%89%88%E6%B6%BC%E6%8B%8C%E8%8A%BD%E8%8F%9C
9.000229835510254
cook_tine is na
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第2218筆資料
壽司醋飯（蒟蒻飯） link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12695031-%E5%A3%BD%E5%8F%B8%E9%86%8B%E9%A3%AF%E8%92%9F%E8%92%BB%E9%A3%AF
8.00090217590332
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第2219筆資料
氣炸雞軟骨 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12701992-%E6%B0%A3%E7%82%B8%E9%9B%9E%E8%BB%9F%E9%AA%A8
7.000174045562744
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第2220筆資料
迷你小鬆餅 Pancake Cereal link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12695172-%E8%BF%B7%E4%BD%A0%E5%B0%8F%E9%AC%86%E9%A4%85-pancak

全聯食譜之爸爸回家做晚飯-韓式泡菜餃子鍋 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12700097-%E5%85%A8%E8%81%AF%E9%A3%9F%E8%AD%9C%E4%B9%8B%E7%88%B8%E7%88%B8%E5%9B%9E%E5%AE%B6%E5%81%9A%E6%99%9A%E9%A3%AF-%E9%9F%93%E5%BC%8F%E6%B3%A1%E8%8F%9C%E9%A4%83%E5%AD%90%E9%8D%8B
5.000458478927612
cook_tine is na
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第2251筆資料
火腿蛋炒飯 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12659350-%E7%81%AB%E8%85%BF%E8%9B%8B%E7%82%92%E9%A3%AF
6.000005483627319
cook_tine is na
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第2252筆資料
筍丁肉圓 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12699938-%E7%AD%8D%E4%B8%81%E8%82%89%E5%9C%93
8.00005578994751
Number_of_copies is alive
寫檔完成
進入寫入圖片前
圖片下載完成
_____________________________________
第2253筆資料
蝦干鹹肉白菜冬粉 link
https://cookpad.com/tw/%E9%A3%9F%E8%AD%9C/12697346-%E8%9D%A6%E5%B9%B2%E9%B9%B9%E8%82%89%E7%99%BD%E8%8F%9C%E5%86%AC%E7%B2%89
13.00048017501831
cook_tine is na
Number_of_copies is alive
寫檔完成
進入寫

In [ ]:
#print(save_next_html)
print(len(save_next_html))




In [ ]:
#print(list_save)
print(len(list_save))
print(list_save[990])



In [ ]:
import pandas as pd
from pandas.core.frame import DataFrame
from datetime import datetime
#包含两个不同的子列表[1,2,3,4]和[5,6,7,8]
data=DataFrame(list_save)#这时候是以行为标准写入的
print(data)

now = datetime.now()
s1 = datetime.strftime(now,'%Y-%m-%d')
s2= datetime.strftime(now,'%H:%M:%S')
s2=s2.replace(":","_")
#print (type(now))
#print (now)
print (s1)
print(s2)
save_file_data_time=s1+"_"+s2
#print(s.replace(" ","")))

data.to_csv('cookpad{}.csv'.format(save_file_data_time),header=False,index=False,encoding='utf-8-sig')

In [ ]:
!pip install pandas